In [ ]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    peft==0.3.0 --quiet

# Installing the Reinforcement Learning library directly from github.
%pip install git+https://github.com/lvwerra/trl.git@25fa1bd

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl: Transformer Reinforcement Learning Library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import evaluate

import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

### Load FLAN-T5 Model, Prepare Reward Model and Toxicity Evaluator

### Load Data and FLAN-T5 Model Fine-Tuned with Summarization Instruction

In [3]:
model_name = "google/flan-t5-base"
huggingface_dataset_name = "knkarthick/dialogsum"

dataset_original = load_dataset(huggingface_dataset_name)

dataset_original

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [4]:
def build_dataset(model_name,
                  dataset_name,
                  input_min_text_length,
                  input_max_text_length):
  # load dataset
  dataset = load_dataset(dataset_name, split = "train")

  # Filter the dialogues of length between input_min_text_length and input_max_text_length characters
  dataset = dataset.filter(lambda x : len(x['dialogue']) > input_min_text_length and len(x['dialogue']) <= input_max_text_length, batched = False)

  # Prepare tokenizer. Setting device_map="auto" allows to switch between GPU and CPU automatically
  tokenizer = AutoTokenizer.from_pretrained(model_name, device_map = "auto")

  def tokenize(sample):
    prompt = f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
    sample['input_ids'] = tokenizer.encode(prompt)
    sample['query'] = tokenizer.decode(sample['input_ids'])
    return sample

  # Tokenizer each dialogue
  dataset = dataset.map(tokenize, batched = False)
  dataset.set_format(type = 'torch')

  # Split the dataset into train and test parts
  dataset_splits = dataset.train_test_split(test_size = 0.2, shuffle = False, seed = 42)

  return dataset_splits

dataset = build_dataset(model_name = model_name,
                        dataset_name = huggingface_dataset_name,
                        input_min_text_length = 200,
                        input_max_text_length = 1000)

print(dataset)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/10022 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 8017
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 2005
    })
})


In [5]:
def print_number_of_trainable_model_parameters(model):
  trainable_model_params = 0
  all_model_params = 0
  for _, param in model.named_parameters():
    all_model_params += param.numel()
    if param.requires_grad:
      trainable_model_params += param.numel()
  return f"\ntrainable model parameteres: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [9]:
lora_config = LoraConfig(
    r = 32,
    lora_alpha = 32,
    target_modules = ["q", "v"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = TaskType.SEQ_2_SEQ_LM # FLAN=T5
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype = torch.bfloat16
)

# The line './peft-dialogue-summary-checkpoint-from-s3/' in your code refers to the local path where the pretrained PEFT (Parameter-Efficient Fine-Tuning) model checkpoint is stored.
# PeftModel.from_pretrained() is being used to load a pretrained model checkpoint from this specified directory.
peft_model = PeftModel.from_pretrained(
    model,
    './peft-dialogue-summary-checkpoint-from-s3/',
    lora_config = lora_config,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    is_trainable = True
)

print(f'PEFT model parameters to be updated:\n{print_number_of_trainable_model_parameters(peft_model)}\n')

In [ ]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(
    peft_model,
    torch_dtype = torch.bfloat16,
    is_trainable = True
)

print(f'PPO model parameters to be updated (ValueHead + 769 params):\n{print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)

'''
PPO model parameters to be updated (ValueHead + 769 params):

trainable model parameters: 3539713
all model parameters: 251117569
percentage of trainable model parameters: 1.41%

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)
'''

In [ ]:
ref_model = create_reference_model(ppo_model)

print(f"Reference model parameters to be updated \n{print_number_of_trainable_model_parameters(ref_model)}\n")

'''
Reference model parameters to be updated:

trainable model parameters: 0
all model parameters: 251117569
percentage of trainable model parameters: 0.00%
'''

### Prepare Reward Model

In [ ]:
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name, device_map = "auto")
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map = 'auto')

print(toxicity_model.config.id2lbel)

'''
{0: 'nothate', 1: 'hate'}
'''